In [8]:
# set module auto-reloaded 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import numpy as np
import dahuffman
import pickle
from Compressible_Huffman import Huffman
from getModel import get_simplemodel



In [33]:
import os
file_path = r'trainedNN'

sz = os.path.getsize(file_path)
print(f'The {file_path} size is', sz, 'bytes')

The trainedNN size is 2848202 bytes


In [34]:
from pathlib import Path

trainedNN = tf.keras.models.load_model('trainedNN.tf')

OSError: Unable to open file (file signature not found)

In [27]:
# Load the CompressibleNN instance
with open('compressed_nn_new.pkl', 'rb') as compressed_nn:
    net_model = tf.keras.models.load_model(r'trainedNN')
    codec = pickle.load(compressed_nn)
    compressed_model_weights = pickle.load(compressed_nn)

# Create a new instance of CompressibleNN
compNN = Huffman(net_model)
compNN.codec = codec

OSError: Unable to open file (file signature not found)

In [7]:
decompressed_weights = compNN.decompressNN(compressed_model_weights)

In [8]:
# Compare the original weights with the decompressed weights
with open('original_model_weights.pkl', 'rb') as file:
    original_model_weights = pickle.load(file)

differences = compNN.compare_weights(original_model_weights, decompressed_weights)

print("Differences between original and decompressed weights:")
for i, diff in enumerate(differences):
    print(f"Layer {i+1}: Max Difference = {np.max(diff)}, Mean Difference = {np.mean(diff)}")


Differences between original and decompressed weights:
Layer 1: Max Difference = 0.0, Mean Difference = 0.0
Layer 2: Max Difference = 0.0, Mean Difference = 0.0
Layer 3: Max Difference = 0.0, Mean Difference = 0.0
Layer 4: Max Difference = 0.0, Mean Difference = 0.0
Layer 5: Max Difference = 0.0, Mean Difference = 0.0
Layer 6: Max Difference = 0.0, Mean Difference = 0.0


In [24]:
from tensorflow.keras.datasets import mnist
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
image_size = x_train.shape[1]
input_size = image_size * image_size

# Preprocess the data (reshape and normalize)
x_train = np.reshape(x_train, [-1, image_size, image_size, 1])  # Keep the shape (None, 28, 28, 1)
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, image_size, image_size, 1])  # Keep the shape (None, 28, 28, 1)
x_test = x_test.astype('float32') / 255

optimizer = tf.optimizers.Adam(learning_rate=1e-3, beta_1=0.9)
compNN.compile(optimizer, loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Assuming your model is already defined and compiled, you can now evaluate it
acc = compNN.evaluate(x_test, y_test, verbose=0)
print('Test Accuracy: %.3f' % (acc * 100.0))

Test Accuracy: 243.961
